# Find population of a US CITY based on
    
    ALAND
    AWATER
    LAT
    LONG
    STATE
    UAtype


steps to train a DNN¶
1. Prepare data
_ clean data
_ vectorize/encode data
_ split data in train,validation and test data
2. Train model
_ create model and layers
_ compile mode(optimizer , metrics, loss)
_ fit model(train data, validation data, epochs, mini batch size)
3. Validate model
_ validate on test data
4. Repeat


In [2]:
import pandas as pd
import numpy as np


In [3]:
#prepare data
city_data = []
#00037	Abbeville, LA Urban Cluster	C	19824	8460	29222871	300497	      11.283	       0.116	 29.967602	 -92.098219  
def parseline(line):
    parts = line.split('\t')
    data={}
    namepart = parts[1].split(',')
    data['NAME']=namepart[0]
    data['STATE']=namepart[1].strip().split()[0].strip()
    data['UAtype']=parts[2]
    data['POP10']= int(parts[3])
    data['ALAND']= int(parts[5])
    data['AWATER']= int(parts[6])
    data['LAT']=float(parts[9])
    data['LONG']=float(parts[10])
    return data
with open('Gaz_ua_national.txt','r') as file:
    city_data = [ parseline(line.strip()) for line in file.readlines()[1:]]

In [4]:
city_data[-1]

{'NAME': 'Zuni Pueblo',
 'STATE': 'NM',
 'UAtype': 'C',
 'POP10': 6744,
 'ALAND': 11876813,
 'AWATER': 0,
 'LAT': 35.071062,
 'LONG': -108.823725}

In [5]:
len(city_data)

3592

In [1]:
df = pd.DataFrame(city_data)
df = df.sample(len(df))
df.head()

NameError: name 'pd' is not defined

In [2]:
def to_numerical(labels):
    numerical_dict={}
    for label in labels:
        if label not in numerical_dict:
            numerical_dict[label]=len(numerical_dict)
    return numerical_dict

def to_one_hot(labels):
    numerical = to_numerical(labels)
    one_hot = np.zeros((len(labels),len(numerical)))
    for i,label in enumerate(labels):
        one_hot[i,numerical[label]]=1
    return np.asarray(one_hot)

In [8]:
type(to_one_hot(df['STATE']))

numpy.ndarray

In [9]:
df.head()

,ALAND,AWATER,LAT,LONG,NAME,POP10,STATE,UAtype
0,29222871,300497,29.967602,-92.098219,Abbeville,19824,LA,C
1,11315197,19786,34.179237,-82.379726,Abbeville,5243,SC,C
2,5363441,13221,44.948612,-90.315875,Abbotsford,3966,WI,C
3,7416616,52732,33.824742,-88.554591,Aberdeen,4666,MS,C
4,33002447,247597,45.463186,-98.471033,Aberdeen,25977,SD,C


In [63]:
#normalixe data
train, validation, test = (0.6,0.2,0.2)
numeric_data = np.asarray(df[['ALAND','AWATER','LAT','LONG']].values)
print(numeric_data[0])
mean=[]
std=[]
def normalize_data(numeric_data,fraction):
    global mean, std
    train_size = int(len(numeric_data)*fraction)
    mean =  numeric_data[:train_size].mean(axis=0)
    std =  numeric_data[:train_size].std(axis=0)
    return (numeric_data - mean)/std
 
numeric_data = normalize_data(numeric_data,train)
print(mean)
std
numeric_data[0]

[ 2.9222871e+07  3.0049700e+05  2.9967602e+01 -9.2098219e+01]
[ 7.63547291e+07  1.77061862e+06  3.79396278e+01 -9.32213446e+01]


array([-0.13612293, -0.13034352, -1.40994277,  0.07490752])

In [70]:
states = to_one_hot(df['STATE'].values)
UA = to_one_hot(df['UAtype'].values)
numeric_data = np.concatenate((numeric_data,states,UA),axis=1)

population = np.asarray(df['POP10'].values).astype(np.float32)
population = house_prices.reshape(len(house_prices),1)
population.shape

(3592, 1)

In [71]:
numeric_data.shape

(3592, 312)

In [78]:
train_f, validation_f, test_f = (0.6,0.2,0.2)
train = int(train_f*len(numeric_data))
validation = int((train_f+validation_f)*len(numeric_data))
print(validation)
x_train,x_validate,x_test = (numeric_data[:train],numeric_data[train:(validation)],numeric_data[validation:])
y_train,y_validate,y_test = (population[:train],population[train:(validation)],population[validation:])

2873


In [79]:
x_train.shape,x_validate.shape,x_test.shape

((2155, 312), (718, 312), (719, 312))